In [1]:
import time
import os


def read_proc_self_stat():
    """Read utime and stime from /proc/self/stat."""
    with open("/proc/self/stat", "r") as f:
        fields = f.read().split()
        utime = int(fields[13])  # 14th field: user mode jiffies
        stime = int(fields[14])  # 15th field: kernel mode jiffies
    return utime, stime


def measure_cpu_utilization(code_block, *args, **kwargs):
    """
    Measure average CPU utilization for a code block (function).
    code_block: function to execute
    args, kwargs: arguments to the function
    Returns: CPU utilization percentage
    """
    clk_tck = os.sysconf(os.sysconf_names["SC_CLK_TCK"])

    # Read initial CPU times and wall time
    utime_start, stime_start = read_proc_self_stat()
    wall_start = time.monotonic()

    # Run the code section
    code_block(*args, **kwargs)

    # Read final CPU times and wall time
    utime_end, stime_end = read_proc_self_stat()
    wall_end = time.monotonic()

    # Calculate CPU time used (in seconds)
    cpu_time = (utime_end - utime_start + stime_end - stime_start) / clk_tck

    # Calculate wall-clock time (in seconds)
    wall_time = wall_end - wall_start

    # Compute CPU utilization as a percentage
    utilization = (cpu_time / wall_time) * 100 if wall_time > 0 else 0.0
    return utilization


# Example usage:
def busy_work():
    # Example: CPU-intensive loop
    for _ in range(10**8):
        pass


util = measure_cpu_utilization(busy_work)
print(f"Average CPU utilization: {util:.2f}%")


Average CPU utilization: 99.66%


In [2]:
!pip list

Package                   Version        Editable project location
------------------------- -------------- -------------------------
alembic                   1.15.2
altair                    5.5.0
annotated-types           0.7.0
anyio                     4.9.0
archspec                  0.2.5
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 3.0.0
async_generator           1.10
async-lru                 2.0.5
attrs                     25.3.0
babel                     2.17.0
beautifulsoup4            4.13.4
bleach                    6.2.0
blinker                   1.9.0
bokeh                     3.7.3
boltons                   24.0.0
Bottleneck                1.5.0
Brotli                    1.1.0
cached-property           1.5.2
certifi                   2025.4.26
certipy                   0.2.2
cffi                      1.17.1
charset-normalizer        3.4.2
click                     8.2.0
cloudpickle            

In [3]:
from code_monitor.instrument_code import SystemMonitor

In [4]:
# Create monitor instance
monitor = SystemMonitor()

# Get current memory usage
print(f"Current RSS memory usage: {monitor.get_current_rss():.2f} MB")

# Take CPU snapshot
before_snapshot = monitor.snapshot_cpu()
print(f"CPU snapshot: {before_snapshot}")

# Simulate some work
for _ in range(10**8):
    pass

# Take another snapshot
after_snapshot = monitor.snapshot_cpu()
print(f"CPU snapshot after work: {after_snapshot}")

print(
    f"\nCPU utilization:{monitor.compute_cpu_usage(before_snapshot, after_snapshot):.2f}%"
)


Current RSS memory usage: 69.96 MB
CPU snapshot: (104, 3, 1402.518335557)
CPU snapshot after work: (257, 3, 1404.051386016)

CPU utilization:99.80%
